In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math

import sys

import numpy as np

import matplotlib.pyplot as plt

import keras

import pandas as pd

import numpy as np

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import LSTM

from keras.layers import Dropout

from keras.layers import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

In [3]:
%cd /content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator

/content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator


In [4]:
df=pd.read_csv("repl_indicator.csv")
df_time = pd.read_csv("time.csv")
print('Number of rows and columns:', df.shape)

time_step = 30
index_rsi = 14
n_days = 15

df

Number of rows and columns: (3962, 25)


,close,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,314.0,5.749393,0.799865,312.0,316.40,327.30,348.750,341.450000,36.000000,-4.273180e+09,...,35.978883,317.911960,326.084877,333.943486,336.766261,-7.751160e+08,405.518690,348.750,291.981310,1.845514e+07
1,314.0,5.749393,0.799865,314.0,316.80,322.60,344.300,341.183333,31.914894,-4.273180e+09,...,33.169562,316.607973,323.887627,332.044107,336.019826,-2.162091e+09,396.968177,344.300,291.631823,4.239395e+07
2,310.0,5.736572,0.789676,314.0,315.00,318.20,340.300,340.383333,21.259843,-4.273507e+09,...,25.975991,314.405316,321.362604,329.944668,335.166717,-1.872136e+09,390.445473,340.300,290.154527,2.564569e+07
3,304.0,5.717028,0.789676,304.0,310.80,314.50,337.150,335.950000,23.893805,-4.274156e+09,...,28.593895,310.936877,318.205767,327.473747,334.144857,-1.979866e+09,388.141331,337.150,286.158669,3.355705e+07
4,302.0,5.710427,0.784481,304.0,308.80,311.30,333.950,333.750000,22.807018,-4.274267e+09,...,33.140244,307.957918,315.259264,325.047676,333.090928,-1.043824e+09,385.349263,333.950,282.550737,1.683587e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,528.8,6.270610,0.669673,528.8,536.30,542.76,529.410,564.648333,59.819820,-1.237088e+06,...,75.667911,533.909313,535.680348,533.930772,559.040900,1.121271e+09,568.802807,529.410,490.017193,5.143446e+07
3958,534.6,6.281519,0.677018,528.8,533.22,542.90,529.955,562.715000,59.269406,-7.534487e+05,...,74.822774,534.139542,535.483921,533.994508,558.239559,8.951193e+08,569.316775,529.955,490.593225,4.409455e+07
3959,548.1,6.306458,0.677018,548.1,532.14,543.73,530.330,561.270000,61.385281,4.092032e+05,...,72.273594,538.793028,537.777753,535.337888,557.907115,1.210879e+09,570.257624,530.330,490.402376,4.604101e+07
3960,543.4,6.297846,0.671213,548.1,536.30,542.55,531.550,559.723333,59.280936,1.082603e+05,...,72.580201,540.328685,538.799980,536.105708,557.431472,7.791105e+08,571.511060,531.550,491.588940,3.509507e+07


In [5]:
df_time = df_time.iloc[:,0]

In [6]:
df_time = df_time.str[:10]

In [7]:
res = []

# training set/ test set

In [8]:
feature = df.iloc[:,1:]

idx = int(feature.shape[0]*0.8)

training_set = feature.iloc[:idx].values
test_set = feature.iloc[idx:].values
training_set.shape

(3169, 24)

In [9]:
training_set

array([[5.74939299e+00, 7.99865419e-01, 3.12000000e+02, ...,
        3.48750000e+02, 2.91981310e+02, 1.84551436e+07],
       [5.74939299e+00, 7.99865419e-01, 3.14000000e+02, ...,
        3.44300000e+02, 2.91631823e+02, 4.23939500e+07],
       [5.73657230e+00, 7.89676050e-01, 3.14000000e+02, ...,
        3.40300000e+02, 2.90154527e+02, 2.56456931e+07],
       ...,
       [7.15773548e+00, 2.37724089e-01, 1.28400000e+03, ...,
        1.32700000e+03, 1.25050026e+03, 5.38986316e+07],
       [7.16472038e+00, 2.39390380e-01, 1.28400000e+03, ...,
        1.32435000e+03, 1.24695484e+03, 3.23220164e+07],
       [7.16162200e+00, 2.39390380e-01, 1.28900000e+03, ...,
        1.31980000e+03, 1.24554411e+03, 5.06227592e+07]])

In [10]:
# 정규화
ss = StandardScaler()

training_set_scaled = ss.fit_transform(training_set)

X_train = []

y_train = []

for i in range(time_step, idx-n_days):
    X_train.append(training_set_scaled[i-time_step:i])

    y_train.append(training_set_scaled[i+n_days, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
y_train

array([-1.25795143, -1.16638752, -1.16638752, ...,  1.70129169,
        1.71258764,  1.70757696])

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_train.shape

(3124, 30, 24)

In [12]:
y_train.shape

(3124,)

# 모델 학습

# 모델이 예상한 값 불러오기

In [13]:
dataset_train = feature.iloc[:idx] 

dataset_test = feature.iloc[idx:]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)

inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_step:].values

#inputs = inputs.reshape(-1,n_features)

inputs = ss.transform(inputs)

X_test = []
for i in range(time_step, dataset_test.shape[0]):

    X_test.append(inputs[i-time_step:i])

X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print(X_test.shape)

(763, 30, 24)


In [14]:
def measure_accuarcy(predict, real=dataset_test.values[:,0], n_days=1):
    predict_result = []
    real_result = []
    count = 0
    result = 0
    correct_up = 0
    correct_down = 0
    real_up = 0
    real_down = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==1:
            real_up += 1
            if predict_result[i] ==1:
                correct_up += 1
        elif real_result[i]==0:
            if predict_result[i] == 0:
                correct_down += 1
                
    count = correct_up + correct_down
    
    return (count/n_predicting_days,correct_up,correct_down)

In [15]:
def measure_diff(predict, real = dataset_test.values[:,0], n_days=1):
    
    predict_result = []
    real_result = []
    result = 0
    predict_diff = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==predict_result[i]:
            predict_diff += abs(predict[i]-real[i])
        else:
            predict_diff -= abs(predict[i]-real[i])
    
    return predict_diff

In [16]:
model13 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model13.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model13.add(LSTM(60))
model13.add(Dropout(0.4))


# Adding the output layer i 
 
model13.add(Dense(units = 1))

# Compiling the RNN

model13.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model13.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 10ms/step - loss: 0.1769
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1142
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0982
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0934
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0771
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0636
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0612
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0583
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0551
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0513
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0509
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0442
Epoch 13/80
98/98 [=============================

In [17]:
predicted_stock_price = model13.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.269587672132145
[45.69008]


In [18]:
model14 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model14.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model14.add(LSTM(60))
model14.add(Dropout(0.8))


# Adding the output layer i 
 
model14.add(Dense(units = 1))

# Compiling the RNN

model14.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model14.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2919
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2011
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1810
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1704
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1542
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1427
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1462
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1314
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1267
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1310
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1172
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1080
Epoch 13/80
98/98 [==============================

In [19]:
predicted_stock_price = model14.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.267427687698379
[42.309933]


In [20]:
model15 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model15.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model15.add(LSTM(100))
model15.add(Dropout(0.4))


# Adding the output layer i 
 
model15.add(Dense(units = 1))

# Compiling the RNN

model15.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model15.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.1738
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1068
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0836
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0817
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0661
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0641
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0521
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0558
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0453
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0439
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0398
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0369
Epoch 13/80
98/98 [==============================

In [21]:
predicted_stock_price = model15.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.345358405305149
[40.640823]


In [22]:
model16 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model16.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model16.add(LSTM(100))
model16.add(Dropout(0.8))


# Adding the output layer i 
 
model16.add(Dense(units = 1))

# Compiling the RNN

model16.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model16.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2618
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1719
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1641
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1399
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1291
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1160
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1143
Epoch 8/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1032
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1021
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1004
Epoch 11/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0929
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0905
Epoch 13/80
98/98 [==============================

In [23]:
predicted_stock_price = model16.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.320000137676473
[33.18412]


In [24]:
model17 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model17.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model17.add(LSTM(60))
model17.add(Dropout(0.8))


# Adding the output layer i 
 
model17.add(Dense(units = 1))

# Compiling the RNN

model17.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model17.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.3139
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2091
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1733
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1597
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1579
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1594
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1497
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1506
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1396
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1295
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1245
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1198
Epoch 13/80
98/98 [==============================

In [25]:
predicted_stock_price = model17.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.374762873367562
[68.71912]


In [26]:
model18 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model18.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model18.add(LSTM(40))
model18.add(Dropout(0.8))


# Adding the output layer i 
 
model18.add(Dense(units = 1))

# Compiling the RNN

model18.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model18.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.3732
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2341
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2128
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2056
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1961
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1797
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1662
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1685
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1470
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1567
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1459
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1421
Epoch 13/80
98/98 [==============================

In [27]:
predicted_stock_price = model18.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.241321702831247
[55.36433]


In [28]:
model19 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model19.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model19.add(LSTM(60))
model19.add(Dropout(0.8))


# Adding the output layer i 
 
model19.add(Dense(units = 1))

# Compiling the RNN

model19.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model19.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.3001
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1937
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1949
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1705
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1574
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1405
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1397
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1392
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1344
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1221
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1208
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1216
Epoch 13/80
98/98 [==============================

In [29]:
predicted_stock_price = model19.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.235802540861705
[25.904423]


In [30]:
model20 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model20.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model20.add(LSTM(40))
model20.add(Dropout(0.8))


# Adding the output layer i 
 
model20.add(Dense(units = 1))

# Compiling the RNN

model20.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model20.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.3790
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2340
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2042
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1926
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1787
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1688
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1639
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1683
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1527
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1682
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1517
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1530
Epoch 13/80
98/98 [==============================

In [31]:
predicted_stock_price = model20.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
6.228440542204934
[32.501453]


In [32]:
model21 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model21.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[-2:])))

model21.add(LSTM(50))
model21.add(Dropout(0.8))


# Adding the output layer i 
 
model21.add(Dense(units = 1))

# Compiling the RNN

model21.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model21.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.3186
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.2179
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1946
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1655
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1640
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1604
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1400
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1403
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1382
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1402
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1407
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1312
Epoch 13/80
98/98 [==============================

In [33]:
predicted_stock_price = model21.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
6.233736193284479
[13.242483]


In [34]:
score = pd.DataFrame(res)
score

,0
0,69.79
1,68.05
2,62.43
3,65.51
4,64.44
5,70.19
6,69.52
7,68.58
8,58.69


In [35]:
score.to_csv('res_5.csv', index=False)